## Data Augmentation

In [1]:
base_path = "/Volumes/Shawn_HDD/PhD/Project/Date/augmentation_from_matlab/Cropped/based_on_truth/x100100y100100z8080"
volumes_path = base_path + "/volumes"
points_path = base_path + "/points"
length_path = base_path + "/length"


In [2]:
import numpy as np

diff_flip = np.load("../Resources/diff_flip.npy")

In [3]:
import numpy as np
import Functions.MyDataset as MyDataset

pat_id = 13
names = MyDataset.get_pat_names()

pat_name = names[pat_id]
pat_volume_left = np.load(f"{volumes_path}/{pat_name}_augVolume_1_cropped_left.npy")
pat_volume_right = np.load(f"{volumes_path}/{pat_name}_augVolume_1_cropped_right.npy")
pat_point_left = np.load(f"{points_path}/{pat_name}_augPoints_1_cropped_left.npy")
pat_point_right = np.load(f"{points_path}/{pat_name}_augPoints_1_cropped_right.npy")
pat_length_left = np.load(f"{length_path}/{pat_name}_augLength_1_cropped_left.npy")
pat_length_right = np.load(f"{length_path}/{pat_name}_augLength_1_cropped_right.npy")

pat_point_left_new = pat_point_left - diff_flip[pat_id, 0:2, :]
pat_point_right_new = pat_point_right - diff_flip[pat_id, 2:4, :]

In [3]:
## load all patients' landmarks to calculate the mean angle
import Functions.MyDataset as MyDataset

pat_names = MyDataset.get_pat_names()

pat_points = []
for pt_name in pat_names:
    pat_point_left = np.load(f"{points_path}/{pt_name}_augPoints_1_cropped_left.npy").reshape((1, 2, 3))
    pat_point_right = np.load(f"{points_path}/{pt_name}_augPoints_1_cropped_right.npy").reshape((1, 2, 3))
    if len(pat_points) == 0:
        pat_points = pat_point_left
        pat_points = np.concatenate((pat_points, pat_point_right), axis=0)
    else:
        pat_points = np.concatenate((pat_points, pat_point_left), axis=0)
        pat_points = np.concatenate((pat_points, pat_point_right), axis=0)

In [32]:
# point_left_diff = np.asarray([[0, 0, 0], [0, 0, 0]])
# point_right_diff = np.asarray([[0, 0, 0], [0, 0, 0]])
point_left_diff = np.asarray([[0, 0, 0], [0, 0, 0]])
point_right_diff = np.asarray([[0, 0, 0], [0, 0, 0]])

# pat_point_left_new = pat_point_left - point_left_diff
# pat_length_left_new = pat_length_left + point_left_diff
# 
# point_right_diff[:, 0] = - point_right_diff[:, 0]
# pat_point_right_new = pat_point_right - point_right_diff
# pat_length_right_new = pat_length_right + point_right_diff

In [6]:
print(pat_point_left)
print(pat_point_left_new)

print(pat_point_right)
print(pat_point_right_new)

In [5]:
import Functions.Visualization as Visualization

import importlib
importlib.reload(Visualization)

Visualization.show_two_landmarks(pat_volume_left, pat_point_left, pat_volume_right, pat_point_right, [0.15, 0.15, 0.15])

Visualization.show_two_landmarks(pat_volume_left, pat_point_left_new, pat_volume_right, pat_point_right_new, [0.15, 0.15, 0.15])

In [9]:
import math

def angle_3D_points(x1, y1, z1,
                    x2, y2, z2,
                    x3, y3, z3):
    # Find direction ratio of line AB
    ABx = x1 - x2;
    ABy = y1 - y2;
    ABz = z1 - z2;

    # Find direction ratio of line BC
    BCx = x3 - x2;
    BCy = y3 - y2;
    BCz = z3 - z2;
    
    v1_mag = math.sqrt(ABx*ABx + ABy*ABy + ABz*ABz)
    v1_norm = (ABx/v1_mag, ABy/v1_mag, ABz/v1_mag)

    v2_mag = math.sqrt(BCx*BCx + BCy*BCy + BCz*BCz)
    v2_norm = (BCx/v2_mag, BCy/v2_mag, BCz/v2_mag)
    
    dot_products = v1_norm[0] * v2_norm[0] + v1_norm[1] * v2_norm[1] + v1_norm[2] * v2_norm[2]
    
    angle = (math.acos(dot_products) * 180.0) / math.pi
    print(round(angle, 2))
    
    return round(angle, 2)


In [10]:
def cal_angles(pt_points, pt_points_new):
    (Ax1, Ay1, Az1) = pt_points[0]
    (Bx2, By2, Bz2) = np.mean(pt_points, axis=0)
    angle_3D_points(Ax1, Ay1, Az1,
                   Bx2, By2, Bz2,
                   Bx2+1, By2, Bz2)
    angle_3D_points(Ax1, Ay1, Az1,
                   Bx2, By2, Bz2,
                   Bx2, By2+1, Bz2)
    angle_3D_points(Ax1, Ay1, Az1,
                   Bx2, By2, Bz2,
                   Bx2, By2, Bz2+1)

    (Ax1_, Ay1_, Az1_) = pt_points_new[0]
    (Bx2_, By2_, Bz2_) = np.mean(pt_points_new, axis=0)
    angle_3D_points(Ax1_, Ay1_, Az1_,
                   Bx2_, By2_, Bz2_,
                   Bx2_+1, By2_, Bz2_)
    angle_3D_points(Ax1_, Ay1_, Az1_,
                   Bx2_, By2_, Bz2_,
                   Bx2_, By2_+1, Bz2_)
    angle_3D_points(Ax1_, Ay1_, Az1_,
                   Bx2_, By2_, Bz2_,
                   Bx2_, By2_, Bz2_+1)

In [34]:
cal_angles(pat_point_left, pat_point_left_new)

In [12]:
def cal_angles_one(pt_points):
    (Ax1, Ay1, Az1) = pt_points[0]
    (Bx2, By2, Bz2) = np.mean(pt_points, axis=0)
    xr = angle_3D_points(Ax1, Ay1, Az1,
                    Bx2, By2, Bz2,
                    Bx2+1, By2, Bz2)
    yr = angle_3D_points(Ax1, Ay1, Az1,
                    Bx2, By2, Bz2,
                    Bx2, By2+1, Bz2)
    zr = angle_3D_points(Ax1, Ay1, Az1,
                    Bx2, By2, Bz2,
                    Bx2, By2, Bz2+1)
    
    return [xr, yr, zr]

In [13]:
angles = []

for i in range(40):
    angle = cal_angles_one(pat_points[i])
    angles.append(angle)
    
angles = np.asarray(angles)

In [42]:
JM_angle = cal_angles_one(pat_point_left)

In [15]:
mean_angle = np.mean(angles, axis=0)

## find way to rotate and translate

In [35]:
from scipy import ndimage

v_left = np.reshape(pat_volume_left, (200, 200, 160))
p_left = np.copy(pat_point_left.reshape((2,3)))

In [154]:
centre = np.asarray([100, 100, 80]) - 1
org = np.mean(pat_point_left, axis=0) - 1
shift_value = np.ceil(centre - org)

In [155]:
v_left_shift = ndimage.shift(v_left, shift_value)
p_left_shift = p_left + shift_value

In [156]:
print(v_left_shift[:, :, 146])

In [157]:
Visualization.show_two_landmarks(v_left_shift, p_left_shift, pat_volume_right, pat_point_right, [0.15, 0.15, 0.15])

In [43]:
[x_angle, y_angle, z_angle] = mean_angle - JM_angle
z_angle = - z_angle
x_angle_r = x_angle * math.pi / 180
y_angle_r = y_angle * math.pi / 180
z_angle_r = z_angle * math.pi / 180

In [27]:
# max rotation angle: 15 Degrees; change to Radian
max_rot_angle = 15
# random rotation angle: -max to +max
rand_angle = 2 * np.random.rand() * max_rot_angle - max_rot_angle
rand_angle_r = rand_angle * math.pi / 180

In [63]:
# v_left_15 = ndimage.rotate(v_left, rand_angle, axes=(0, 1), reshape=False)
# v_left_15 = ndimage.rotate(v_left_15, rand_angle, axes=(0, 2), reshape=False)
# v_left_15 = ndimage.rotate(v_left_15, rand_angle, axes=(1, 2), reshape=False)

v_left_15 = ndimage.rotate(v_left, x_angle, axes=(0, 1), reshape=False)
# v_left_15 = ndimage.rotate(v_left_15, y_angle, axes=(0, 2), reshape=False)
# v_left_15 = ndimage.rotate(v_left_15, z_angle, axes=(1, 2), reshape=False)

In [64]:
import numpy as np
import math

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians (Euler-Rodrigues formula).
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

org_centre = np.asarray([200, 200, 160]) / 2.
rot_centre = np.asarray(v_left_15.shape) / 2.

v = p_left - org_centre

axis_0 = np.asarray([100, 100, 70]) - org_centre
p_new = np.dot(rotation_matrix(axis_0, x_angle_r), v.T)

# axis_1 = np.asarray([110, 100, 80]) - org_centre
# p_new = np.dot(rotation_matrix(axis_1, y_angle_r), p_new)

# axis_2 = np.asarray([100, 90, 80]) - org_centre
# p_new = np.dot(rotation_matrix(axis_2, z_angle_r), p_new)


p_new = p_new.T + rot_centre
print(p_new)
print(cal_angles_one(p_new))

In [65]:
Visualization.show_two_landmarks(v_left_15, p_new, pat_volume_right, pat_point_right, [0.15, 0.15, 0.15])

In [203]:
cal_angles(pat_point_left, p_new)

## Check the relations between the max err and the angles

In [1]:
import numpy as np

base_dir = "/Users/achs/PhD/code/CT-MRI_LandmarkDetection/Spartan/trained_models"

Y_test_gt = np.load(f"{base_dir}/cropped_dataset/straight_model/one_landmark/100x100x100/no_trans_50aug_6medium/y_test_gt.npy")
Y_test_pr = np.load(f"{base_dir}/cropped_dataset/straight_model/one_landmark/100x100x100/no_trans_50aug_6medium/bestVal_straight_model_cropped_100x100x100_y_test.npy")

In [3]:
X_test = np.load(f"{base_dir}/cropped_dataset/straight_model/one_landmark/100x100x100/no_trans_50aug_6medium/x_test.npy")

In [4]:
err_diff = Y_test_pr - Y_test_gt

In [5]:
err_diff_p2 = np.power(err_diff, 2)
sum_square_err_diff = np.sum(err_diff_p2, axis= (1,2) )
err_dis = np.power(sum_square_err_diff, 1/2) * 0.15
min_err_idx = np.argmin(err_dis, axis=0)
max_err_idx = np.argmax(err_dis, axis=0)
print(f"Min[{min_err_idx}]: {err_dis[min_err_idx]}")
print(f"Max[{max_err_idx}]: {err_dis[max_err_idx]}")
print(f"Mean: {np.mean(err_dis)}")

In [37]:
err_dis_pat = np.reshape(err_dis, (4, 100))

mean_pat = np.mean(err_dis_pat, axis=1)
std_pat = np.std(err_dis_pat, axis=1)

print(f"Mean: {mean_pat}")
print(f"Std: {std_pat}")

for index in range(len(mean_pat)):
    print(f"Test Patient {index}, 95.44%, ({mean_pat[index]-2*std_pat[index]}, {mean_pat[index]+2*std_pat[index]})")

In [9]:
err_max2min_ids = sorted(range(len(err_dis)), key=lambda k: err_dis[k], reverse=True)

In [10]:
print(err_max2min_ids[0:100])

In [32]:
import Functions.MyDataset as MyDataset

import importlib
importlib.reload(MyDataset)

ck_id = 300

pt_name, pt_aug_id = MyDataset.get_pat_from_test_idx(ck_id, split=True, aug_num=50)

In [33]:
ck_volume = X_test[ck_id, :, :, :, 0]
ck_y_gt = Y_test_gt[ck_id, :, :]
ck_y_pr = Y_test_pr[ck_id, :, :]

print(err_dis[ck_id])
print(ck_y_gt)
print(ck_y_pr)

In [34]:
import Functions.Visualization as Visualization

import importlib
importlib.reload(Visualization)

px_d = [0.15, 0.15, 0.15]

Visualization.show_one_landmark_half_volume(ck_volume, ck_y_gt, px_d)

Visualization.show_one_landmark_half_volume(ck_volume, ck_y_pr, px_d)

## check the Augmentation results

In [31]:
import numpy as np
import h5py

data_file_path = "F:/Data/augmentation_exp/original_divided_augmentation/JM_aug_2.mat"

data_file = h5py.File(data_file_path, 'r')

left_vol = np.array(data_file.get('augLeftVol')).T
left_pts = np.array(data_file.get('augLeftPts')).T

right_vol = np.array(data_file.get('augRightVol')).T
right_pts = np.array(data_file.get('augRightPts')).T

data_file.close()

In [32]:
import Functions.Visualization as Visualization

import importlib
importlib.reload(Visualization)

Visualization.show_two_landmarks_half_volume(left_vol, left_pts, [0.15, 0.15, 0.15])
Visualization.show_two_landmarks_half_volume(right_vol, right_pts, [0.15, 0.15, 0.15])

check augmentation results from python's patch aug code

In [2]:
import numpy as np

base_dir = "/Volumes/my_HDD/augmentation_exp/patch_augmentation_python"
volume_path = f"{base_dir}/AH_volume_patch_aug_1.npy"
points_path = f"{base_dir}/AH_points_aug_1.npy"

aug_volumes = np.load(volume_path)
aug_points = np.load(points_path)

print(aug_volumes.shape)
print(aug_points.shape)

In [17]:
import Functions.Visualization as Visualization

ck_aug_id = 99

ck_aug_volume = aug_volumes[ck_aug_id, :, :, :, 0]
ck_aug_points = aug_points[ck_aug_id]

Visualization.show_two_landmarks_half_volume(ck_aug_volume, ck_aug_points, [0.15, 0.15, 0.15])